In [1]:
%matplotlib inline

In [2]:
import os 
import sys
import json
import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt
import neurokit as nk
import scipy as sc
import math 
import scipy.signal as ss
import warnings

from scipy import signal
warnings.filterwarnings('ignore')

In [3]:
path = "/mnt/shared_drive/data/brain_therapy/Brain_Therapy_MVP_Content_Pilot/FA_corrected_signals"
files = os.listdir(path)

Taking all participant signal dictionary and putting them in one dataframe

In [4]:
sensor_data = pd.DataFrame([])
for file in files:
    data = pd.read_csv(os.path.join(path,file))
    data['Participant'] = file.split("-")[3].split(".")[0]
    data['Datetime'] = pd.to_datetime(data['Datetime'])
    data.set_index(['Participant','Datetime'], inplace = True)
    sensor_data = pd.concat([sensor_data,data])

In [5]:
participants = sensor_data.index.get_level_values('Participant').unique().values
participants = sorted(participants)
sensor_data.sort_index(sort_remaining = True, inplace = True)
sensor_data.head()

RSP       PPG     1-SKTA       ECG  \
Participant Datetime                                                           
0001        2019-07-10 15:22:31.000 -1.263123  0.003052  12.941162 -0.234833   
            2019-07-10 15:22:31.004 -1.261902  0.003662  12.945435 -0.226593   
            2019-07-10 15:22:31.008 -1.264648  0.002136  12.946045 -0.215607   
            2019-07-10 15:22:31.012 -1.261902  0.003662  12.945435 -0.206604   
            2019-07-10 15:22:31.016 -1.262207  0.003357  12.946045 -0.198364   

                                          EDA  
Participant Datetime                           
0001        2019-07-10 15:22:31.000  0.041124  
            2019-07-10 15:22:31.004  0.042649  
            2019-07-10 15:22:31.008  0.044175  
            2019-07-10 15:22:31.012  0.039598  
            2019-07-10 15:22:31.016  0.042649

Now, let's load the log events file which contains the timestamps for the events.

In [6]:
log_events = joblib.load("/mnt/shared_drive/data/brain_therapy/log_events.pkl")
log_events.set_index("time", append = True, inplace = True)
log_events = log_events.reset_index().rename(columns = {"participant":"Participant",'type':'Type','time':'Datetime'}).\
                        set_index(['Participant','Type','Datetime'])
log_events = log_events.reset_index('Type')
log_events.head()

Type           context
Participant Datetime                                  
0001        2019-07-10 15:25:32   2D  StartFrontBuffer
            2019-07-10 15:27:32   2D   StopFrontBuffer
            2019-07-10 15:27:32   2D           StartLL
            2019-07-10 15:29:42   2D            StopLL
            2019-07-10 15:29:42   2D           StartHH

### ECG

Loading the curated ECG peaks

In [7]:
good_peaks = joblib.load("/mnt/shared_drive/data/brain_therapy/MVP_Focused_Attention/FA_r_peaks_index.pkl")

Computing the ECG Heart Rate

In [8]:
def get_rmssd(x):
    ans = np.sqrt(np.mean([i**2 for i in x]))
    return ans

In [9]:
feature_df = pd.DataFrame([])
time_differences_df = pd.DataFrame([])
for participant in participants :
    df = pd.DataFrame([])
    data = sensor_data.loc(axis = 0)[participant,:,:,:]
    
    #Taking the clean ECG peaks of the participant
    ecg_peaks = good_peaks["BT-MVP-FA-{}".format(participant)]
    time_stamps =  data.index.get_level_values('Datetime')[ecg_peaks]
    
    #Computing for the time differences of every peak
#     time_differences1 = [(t - s) for s, t in zip(time_stamps,time_stamps[1:])]
    time_differences  = np.diff(time_stamps)/np.timedelta64(1,'ms')
#     time_differences1 = [(x.seconds*1000) + (x.microseconds/1000) for x in time_differences1]
    
#     display(time_differences1, time_differences)
    #Creating a dataframe of time_differences for easy cleaning and other future use
    df = pd.DataFrame(time_stamps[1:].ravel(), columns = ['Datetime'])
    df['time_differences'] = time_differences
    
    #Taking out time differences that is more than a minute
    df = df[df.time_differences < 60000]
    diff_mean = np.mean(df.time_differences)
    upper_threshold = diff_mean + (3*np.std(df.time_differences))   
    lower_threshold = diff_mean - (3*np.std(df.time_differences))
    df = df[(df.time_differences<upper_threshold) & (df.time_differences>lower_threshold)]
    
    #Interpolating the time differences and getting the values every two seconds
    df.set_index('Datetime', inplace = True)
    df = df.time_differences.resample('4ms').interpolate(method = 'linear')
    features = pd.DataFrame(df.resample('2s').first())
    
    features['Participant'] = participant
    
    #Solving for Heart Rate
    features['ecg_heart_rate'] = (60/features['time_differences'])*1000
    
    #Solving for ECG HRV SDNN
    features['ecg_hrv_std'] = features[['ecg_heart_rate']].rolling('30s').std()
    features['ecg_hrv_std'] = features['ecg_hrv_std'].resample('2s').first()
    
    #Solving for ECG HRV RMSSD
    features['ecg_hrv_rmssd'] = features[['time_differences']].rolling('30s', min_periods = 2).apply(lambda x: get_rmssd(x))
    features['ecg_hrv_rmssd'] = features['ecg_hrv_rmssd'].resample('2s').first()
    
    #Solving for ECG LnRMSSD
    features['ecg_hrv_lnrmssd'] = features[['ecg_hrv_rmssd']].apply(lambda x: np.log(x), axis = 1)
    
    features = features.reset_index().set_index(['Participant','Datetime'])
    time_differences_df = pd.concat([time_differences_df,df])
    feature_df = pd.concat([feature_df,features])
#     break
feature_df.drop('time_differences',axis = 1, inplace = True)

In [10]:
feature_df.head()

ecg_heart_rate  ecg_hrv_std  ecg_hrv_rmssd  \
Participant Datetime                                                          
0001        2019-07-10 15:22:32       56.390977          NaN            NaN   
            2019-07-10 15:22:34       58.689424     1.625247    1043.373394   
            2019-07-10 15:22:36       58.843168     1.373544    1035.529128   
            2019-07-10 15:22:38       61.939107     2.277549    1019.231144   
            2019-07-10 15:22:40       61.907033     2.370811    1009.422396   

                                 ecg_hrv_lnrmssd  
Participant Datetime                              
0001        2019-07-10 15:22:32              NaN  
            2019-07-10 15:22:34         6.950214  
            2019-07-10 15:22:36         6.942668  
            2019-07-10 15:22:38         6.926804  
            2019-07-10 15:22:40         6.917134

### PPG

In [11]:
def low_pass_filter(x, cut_off, order = 6, sampling_rate = 250):
    """
    This is a butterworth low pass filter.
    input:
        x - 1d array signal to be filtered.
        f_c_low - low pass filter cut off frequency in fraction of nyquist rate.
    return:
        filtered_x - filtered signal
    """
    nyquist = (cut_off/sampling_rate/2)
    # nth order Butterworth low pass filter at corner frequency
    b, a = ss.butter(order, nyquist, btype='low')
    filtered_x = ss.filtfilt(b, a, x)
    return filtered_x

In [12]:
def window_index_pairs(fast, slow):
    """
    computes the index of window span based on the intersection
    of 2 input signals.
    input:
        fast - 1d array high frequency signal
        slow - 1d array low frequenct signal
    return:
        window - list of span indices
    """
    positive_crossing = []
    negative_crossing = []
    for i in range(len(fast)-1):
        if fast[i] <= slow[i] and fast[i+1] > slow[i+1]:
            positive_crossing.append(i)
        if fast[i] >= slow[i] and fast[i+1] < slow[i+1] and len(positive_crossing)>0:
            negative_crossing.append(i)

    window = list(zip(positive_crossing,negative_crossing))
    return window


In [13]:
def get_peak(signal, fast_cor_f = 0.2, slow_cor_f = 0):
    """
    computes the indices of glucose peaks.
    input:
        egv - 1d array filtered egv signal.
        fast_cor_f - low pass filter higher cut-off frequency in fraction of nyquist rate.
        slow_cor_f - low pass filter lower cut-off frequency in fraction of nyquist rate.
    return:
        peak_index - 1d array of indices of glucose peaks.
        window_span - List of span indices. Inside of which a glucose peak is located.
        lpf_fast - higher cut-off frequency low pass filtered signal.
        lpf_slow - lower cut-off frequency low pass filtered signal.
    """
    signal_dc = np.mean(signal)
    signal_zero = signal - signal_dc
    lpf_fast = low_pass_filter(signal_zero, fast_cor_f)
    if slow_cor_f == 0:
        lpf_slow = np.zeros(len(signal))
    else:
        lpf_slow = low_pass_filter(signal_zero, slow_cor_f)
    window_span = window_index_pairs(lpf_fast, lpf_slow)
    peak_index =  np.array([start+np.argmax(signal_zero[start:end]) for start, end in window_span])

    return peak_index, window_span, lpf_fast + signal_dc, lpf_slow + signal_dc

Generating Heart Rate from PPG

In [17]:
heart_rate_df = pd.DataFrame([])
peaks_dict = {}
for participant in participants:
    data = sensor_data.loc(axis = 0)[participant]
    ppg_peaks,_,_,_ = get_peak(data['PPG'].values, 6, 4)
    timestamps =  data.index.get_level_values('Datetime')[ppg_peaks]
    
    #Saving the peaks fpr future use
    peaks_dict[participant] = [timestamps, ppg_peaks]
    
    #Getting the time differences every peak to peak
    time_differences  = np.diff(timestamps)/np.timedelta64(1,'ms')
    

    df = pd.DataFrame(timestamps[1:].ravel(), columns = ['time'])
    df['time_differences'] = time_differences
    
    #Removing peak distances that exceeds the threshold
    df = df[df.time_differences < 60000]
    diff_mean = np.mean(df.time_differences)
    upper_threshold = diff_mean + (3*np.std(df.time_differences))   
    lower_threshold = diff_mean - (3*np.std(df.time_differences))
    df = df[(df.time_differences<upper_threshold) & (df.time_differences>lower_threshold)]
    
    #Interpolating and taking the value every 2 seconds
    df.set_index('time', inplace = True)
    df_resampled =df['time_differences'].resample('4ms')
    interpolated = df_resampled.interpolate(method = 'linear')
    values = interpolated.resample('2s').first()
    
    df = pd.DataFrame(values)
    df['Participant'] = participant
    heart_rate_df = pd.concat([heart_rate_df,df])
heart_rate_df = heart_rate_df.reset_index().set_index(['Participant','time'])

In [18]:
# feature_df['ppg_heart_rate'] = (60/heart_rate_df['time_differences'][:-1])*1000
# feature_df.head()

Solving for PPG Amplitude

In [19]:
amplitudes_df =pd.DataFrame([])
for participant in participants:
    data = sensor_data.loc(axis = 0)[participant,:]
    ppg = data[['PPG']]
    values = data[['PPG']].values
    peak_df = pd.DataFrame(values[peaks_dict[participant][1]],columns = ['ppg_amplitude'], index = peaks_dict[participant][0])
    peak_df = peak_df.groupby(pd.Grouper(freq = '2s')).mean()
    
    #RECENTLY ADDED
    mean = np.mean(peak_df.ppg_amplitude)
    upper_threshold = mean + (3*np.std(peak_df.ppg_amplitude))   
    lower_threshold = mean - (3*np.std(peak_df.ppg_amplitude))
    peak_df = peak_df[(peak_df.ppg_amplitude<upper_threshold) & (peak_df.ppg_amplitude>lower_threshold)]
    
    
    peak_df['Participant'] = participant    
    amplitudes_df = pd.concat([amplitudes_df,peak_df])
amplitudes_df = amplitudes_df.reset_index().set_index(['Participant','Datetime'])

In [20]:
feature_df = feature_df.merge(amplitudes_df,how = 'left',left_index = True, right_index = True)
feature_df.head()

ecg_heart_rate  ecg_hrv_std  ecg_hrv_rmssd  \
Participant Datetime                                                          
0001        2019-07-10 15:22:32       56.390977          NaN            NaN   
            2019-07-10 15:22:34       58.689424     1.625247    1043.373394   
            2019-07-10 15:22:36       58.843168     1.373544    1035.529128   
            2019-07-10 15:22:38       61.939107     2.277549    1019.231144   
            2019-07-10 15:22:40       61.907033     2.370811    1009.422396   

                                 ecg_hrv_lnrmssd  ppg_amplitude  
Participant Datetime                                             
0001        2019-07-10 15:22:32              NaN            NaN  
            2019-07-10 15:22:34         6.950214            NaN  
            2019-07-10 15:22:36         6.942668       0.990295  
            2019-07-10 15:22:38         6.926804       0.976715  
            2019-07-10 15:22:40         6.917134       0.998230

### Skin Temperature

In [33]:
curated_signals = joblib.load("/mnt/shared_drive/data/brain_therapy/Brain_Therapy_MVP_Content_Pilot/FA_corrected_signals_amplitude.pkl")

In [34]:
curated_signals.keys()

dict_keys(['BT-MVP-FA-0001', 'BT-MVP-FA-0002', 'BT-MVP-FA-0003', 'BT-MVP-FA-0004', 'BT-MVP-FA-0005', 'BT-MVP-FA-0006', 'BT-MVP-FA-0007', 'BT-MVP-FA-0008', 'BT-MVP-FA-0010', 'BT-MVP-FA-0011', 'BT-MVP-FA-0012', 'BT-MVP-FA-0013', 'BT-MVP-FA-0014', 'BT-MVP-FA-0015', 'BT-MVP-FA-0016', 'BT-MVP-FA-0017', 'BT-MVP-FA-0018', 'BT-MVP-FA-0019', 'BT-MVP-FA-0020', 'BT-MVP-FA-0021'])

In [35]:
p = curated_signals.keys()
curated_df = pd.DataFrame([])
for part in p:
    data = curated_signals[part]
    data['Participant'] = part.split("-")[3]
    curated_df = pd.concat([curated_df,data])
curated_df.set_index(["Participant","Datetime"], inplace = True)
curated_df.head()

RSP       PPG     1-SKTA       ECG  \
Participant Datetime                                                           
0001        2019-07-10 15:22:31.000 -1.263123  0.003052  12.941162 -0.234833   
            2019-07-10 15:22:31.004 -1.261902  0.003662  12.945435 -0.226593   
            2019-07-10 15:22:31.008 -1.264648  0.002136  12.946045 -0.215607   
            2019-07-10 15:22:31.012 -1.261902  0.003662  12.945435 -0.206604   
            2019-07-10 15:22:31.016 -1.262207  0.003357  12.946045 -0.198364   

                                          EDA  
Participant Datetime                           
0001        2019-07-10 15:22:31.000  0.041124  
            2019-07-10 15:22:31.004  0.042649  
            2019-07-10 15:22:31.008  0.044175  
            2019-07-10 15:22:31.012  0.039598  
            2019-07-10 15:22:31.016  0.042649

In [36]:
skin_temp_df = pd.DataFrame([])
for participant in participants:
    data = curated_df.loc(axis = 0)[participant,:]
    skin_temp = data[['1-SKTA']]
    skin_x = skin_temp.index.get_level_values('Datetime')
    skin_y = skin_temp['1-SKTA'].values
    
    skt = skin_temp.reset_index(['Participant'], drop = True).groupby(pd.Grouper(freq = '2s')).mean()
    
    mean =  (skt["1-SKTA"].mean())
    upper_threshold = mean + (3*np.std(skt["1-SKTA"]))   
    lower_threshold = mean - (3*np.std(skt["1-SKTA"]))
    skt = skt[(skt["1-SKTA"]<upper_threshold) & (skt["1-SKTA"]>lower_threshold)]
    
    
    skt['Participant'] = participant
    skin_temp_df = pd.concat([skin_temp_df, skt])
skin_temp_df = skin_temp_df.reset_index().set_index(['Participant','Datetime']) 
skin_temp_df.rename(columns = {"1-SKTA":"skt_skin_temp"}, inplace = True)

In [37]:
#Removing Skin Temperature outliers
skin_temp_df.head()

skt_skin_temp
Participant Datetime                          
0001        2019-07-10 15:22:30      12.951174
            2019-07-10 15:22:32      12.956869
            2019-07-10 15:22:34      12.956752
            2019-07-10 15:22:36      12.956626
            2019-07-10 15:22:38      12.956655

In [38]:
feature_df= feature_df.merge(skin_temp_df, how = 'left', left_index = True, right_index = True)
feature_df.head()

ecg_heart_rate  ecg_hrv_std  ecg_hrv_rmssd  \
Participant Datetime                                                          
0001        2019-07-10 15:22:32       56.390977          NaN            NaN   
            2019-07-10 15:22:34       58.689424     1.625247    1043.373394   
            2019-07-10 15:22:36       58.843168     1.373544    1035.529128   
            2019-07-10 15:22:38       61.939107     2.277549    1019.231144   
            2019-07-10 15:22:40       61.907033     2.370811    1009.422396   

                                 ecg_hrv_lnrmssd  ppg_amplitude  skt_skin_temp  
Participant Datetime                                                            
0001        2019-07-10 15:22:32              NaN            NaN      12.956869  
            2019-07-10 15:22:34         6.950214            NaN      12.956752  
            2019-07-10 15:22:36         6.942668       0.990295      12.956626  
            2019-07-10 15:22:38         6.926804       0.976715      12.956655  
            2019-07-10 15:22:40         6.917134       0.998230      12.956718

### RSP

In [39]:
def rsp_low_pass_filter(sensor_data, sampling_frequency_hz = 250, corner_frequency_hz = 1.0):
    """
    Filters sensor signals using the second order butterworth filter
    
    Inputs:
        sensor_data (series) : sensor signals
        sampling_frequency_hz (int) : sampling frequency
        corner_frequency_hc (int) : corner frequency
    Output:
        filtered (list) : filtered sensor signal
    """
    order = 5
    nyquist = sampling_frequency_hz/ 2.0
    f_c = np.array([corner_frequency_hz, ], dtype=np.float64)
    f_c /= nyquist
    b, a = signal.butter(order, f_c, btype='low')
    filtered = signal.filtfilt(b, a, sensor_data)
    return filtered

In [40]:
rsp_time_difference = pd.DataFrame([])
for participant in participants:
    data = sensor_data.loc(axis = 0)[participant,:]
    rsp = data[['RSP']]
    rsp['RSP'] = rsp_low_pass_filter(rsp['RSP'])
    rsp_x = rsp.index.get_level_values('Datetime')
    rsp_y = rsp['RSP'].values
    features = nk.rsp_process(data['RSP'],sampling_rate = 250)
    onsets = features['RSP']['Cycles_Onsets']
    onset_times = rsp_x[onsets]
#      [((t - s).microseconds)/1000 for s, t in zip(time_stamps,time_stamps[1:])]
    rsp_peaks = [list(rsp_y).index(np.max(rsp_y[s:t])) for s, t in zip(onsets,onsets[1:])]
    
    time_stamps =  data.index.get_level_values('Datetime')[rsp_peaks]
    
    #Computing the differences between each peak
    time_differences  = np.diff(time_stamps)/np.timedelta64(1,'ms')
    
    
    df = pd.DataFrame(time_stamps[1:].ravel(), columns = ['Datetime'])
    df.set_index('Datetime', inplace = True)
    df['time_differences'] = time_differences
    
    #Removing peak distances that exceeds the threshold
    df = df[df.time_differences < 60000]
    diff_mean = np.mean(df.time_differences)
    upper_threshold = diff_mean + (3*np.std(df.time_differences))   
    lower_threshold = diff_mean - (3*np.std(df.time_differences))
    df = df[(df.time_differences<upper_threshold) & (df.time_differences>lower_threshold)]
    
    rsp_rate = pd.DataFrame(df.resample('4ms').interpolate(method = 'linear'))
    rsp_rate = rsp_rate.resample('2s').first()
    rsp_rate['rsp_rate'] = (60/rsp_rate['time_differences'][:-1])*1000
    rsp_rate['Participant'] = participant
    rsp_rate = rsp_rate.reset_index().set_index(['Participant','Datetime'])
    rsp_time_difference = pd.concat([rsp_time_difference,rsp_rate])
rsp_time_difference.drop('time_differences',axis = 1, inplace = True)

In [41]:
feature_df = feature_df.merge(rsp_time_difference, how = 'left', right_index = True, left_index = True)

In [42]:
feature_df.head()

ecg_heart_rate  ecg_hrv_std  ecg_hrv_rmssd  \
Participant Datetime                                                          
0001        2019-07-10 15:22:32       56.390977          NaN            NaN   
            2019-07-10 15:22:34       58.689424     1.625247    1043.373394   
            2019-07-10 15:22:36       58.843168     1.373544    1035.529128   
            2019-07-10 15:22:38       61.939107     2.277549    1019.231144   
            2019-07-10 15:22:40       61.907033     2.370811    1009.422396   

                                 ecg_hrv_lnrmssd  ppg_amplitude  \
Participant Datetime                                              
0001        2019-07-10 15:22:32              NaN            NaN   
            2019-07-10 15:22:34         6.950214            NaN   
            2019-07-10 15:22:36         6.942668       0.990295   
            2019-07-10 15:22:38         6.926804       0.976715   
            2019-07-10 15:22:40         6.917134       0.998230   

                                 skt_skin_temp   rsp_rate  
Participant Datetime                                       
0001        2019-07-10 15:22:32      12.956869        NaN  
            2019-07-10 15:22:34      12.956752        NaN  
            2019-07-10 15:22:36      12.956626  22.865854  
            2019-07-10 15:22:38      12.956655  27.712759  
            2019-07-10 15:22:40      12.956718  21.755830

### EDA

In [43]:
def get_eda_features(eda):
    if len(eda > 15):
        try:
            features = nk.eda_process(eda = eda, sampling_rate = 250)
        except:
            print("No peaks here")
            return 0
    else:
        return 0
    peaks = len(features['EDA']['SCR_Peaks_Indexes'])
    return peaks

In [44]:
eda_peaks_df = pd.DataFrame([])
for participant in participants:
    data = sensor_data.loc(axis = 0)[participant]
    d = data['EDA'].groupby(pd.Grouper(freq = '60s')).apply(lambda x: get_eda_features(x))
    d = pd.DataFrame(d)
    d = d.resample('4ms').interpolate(method = 'linear').resample('2s').first()
    d['Participant'] = participant
    eda_peaks_df = pd.concat([eda_peaks_df,d])
eda_peaks_df = eda_peaks_df.reset_index().set_index(['Participant','Datetime'])
eda_peaks_df.rename(columns = {"EDA":'eda_no_of_peaks'}, inplace = True)   

No peaks here
No peaks here


In [45]:
feature_df = feature_df.merge(eda_peaks_df, how = 'left', right_index = True, left_index = True)

In [46]:
feature_df.head()

ecg_heart_rate  ecg_hrv_std  ecg_hrv_rmssd  \
Participant Datetime                                                          
0001        2019-07-10 15:22:32       56.390977          NaN            NaN   
            2019-07-10 15:22:34       58.689424     1.625247    1043.373394   
            2019-07-10 15:22:36       58.843168     1.373544    1035.529128   
            2019-07-10 15:22:38       61.939107     2.277549    1019.231144   
            2019-07-10 15:22:40       61.907033     2.370811    1009.422396   

                                 ecg_hrv_lnrmssd  ppg_amplitude  \
Participant Datetime                                              
0001        2019-07-10 15:22:32              NaN            NaN   
            2019-07-10 15:22:34         6.950214            NaN   
            2019-07-10 15:22:36         6.942668       0.990295   
            2019-07-10 15:22:38         6.926804       0.976715   
            2019-07-10 15:22:40         6.917134       0.998230   

                                 skt_skin_temp   rsp_rate  eda_no_of_peaks  
Participant Datetime                                                        
0001        2019-07-10 15:22:32      12.956869        NaN              2.0  
            2019-07-10 15:22:34      12.956752        NaN              2.0  
            2019-07-10 15:22:36      12.956626  22.865854              2.0  
            2019-07-10 15:22:38      12.956655  27.712759              2.0  
            2019-07-10 15:22:40      12.956718  21.755830              2.0

In [47]:
def compute_power_spectra(signal, band):
    try:
        x = np.fft.fft(signal - np.nanmean(signal))
        x = np.abs(x)
        freq = np.fft.fftfreq(len(signal), d = 0.004)
    except:
        return 0
    low, high = np.array(band)
    vals = [i for i in x if high>i>low]
    idx = [list(x).index(i) for i in vals]

    frequencies = freq[idx]
    power = np.sum((frequencies/len(signal))**2)
    return power


In [48]:
bands = {'low_power_spectrum':[0.045, 0.15],'high_power_spectrum':[0.15,0.25]}
eda_powers_df = pd.DataFrame([])
for participant in participants:
    data = curated_df.loc(axis = 0)[participant]
    eda_powers = pd.DataFrame([])
    for index, band in bands.items():
        eda_powers[index] = data['EDA'].groupby(pd.Grouper(freq = '60s')).apply(lambda x: compute_power_spectra(x, band))
    
    eda_powers = eda_powers.resample('4ms').interpolate()
    eda_powers = eda_powers.resample('2s').first()    
    eda_powers['Participant'] = participant
    eda_powers_df = pd.concat([eda_powers_df,eda_powers])


In [49]:
eda_powers_df = eda_powers_df.reset_index().set_index(["Participant","Datetime"])

In [50]:
eda_powers_df.head()

low_power_spectrum  high_power_spectrum
Participant Datetime                                                    
0001        2019-07-10 15:22:00            0.000000             0.000000
            2019-07-10 15:22:02            0.000535             0.001179
            2019-07-10 15:22:04            0.001071             0.002359
            2019-07-10 15:22:06            0.001606             0.003538
            2019-07-10 15:22:08            0.002141             0.004718

In [51]:
feature_df = feature_df.merge(eda_powers_df, how = "left", right_index = True, left_index = True)

In [52]:
feature_df['eda_high_low_ratio'] = feature_df['low_power_spectrum']/feature_df['high_power_spectrum']

In [53]:
eda_mean_df = pd.DataFrame([])
for participant in participants:
    data = sensor_data.loc(axis = 0)[participant]
    mean_df = data[['EDA']].groupby(pd.Grouper(freq = '2s')).mean()
    mean_df['Participant'] = participant
    eda_mean_df = pd.concat([eda_mean_df, mean_df])
eda_mean_df = eda_mean_df.reset_index().set_index(['Participant','Datetime'])

In [54]:
feature_df = feature_df.merge(eda_mean_df, how = "left", right_index = True, left_index = True)

In [55]:
feature_df.rename(columns = {"low_power_spectrum":"eda_low_power_spectrum","high_power_spectrum":"eda_high_power_spectrum",
                            "EDA":"eda_mean"}, inplace = True)

In [56]:
temp1 = feature_df.copy()

## Merging with 2D or 3D labels

In [57]:
log_events.set_index('Type',append = True, inplace = True)

In [58]:
type_times_df = pd.DataFrame([])
for participant in sorted(participants):
    data = log_events.loc(axis = 0)[participant,:,:]
    features = feature_df.loc(axis = 0)[participant]
    two_d = data.loc(axis = 0)[:,:,'2D']
    two_start = two_d[two_d['context'] == 'StartFrontBuffer'].index.get_level_values('Datetime').values[0]
    two_end = two_d[two_d['context'] == 'StopEndBuffer'].index.get_level_values('Datetime').values[0]
    
    three_d = data.loc(axis = 0)[:,:,'3D']
    three_start = three_d[three_d['context'] == 'StartFrontBuffer'].index.get_level_values('Datetime').values[0]
    three_end = three_d[three_d['context'] == 'StopEndBuffer'].index.get_level_values('Datetime').values[0]
    
    time_stamps = feature_df.index.get_level_values('Datetime')
    
    f_temp = features.copy().reset_index()
    times_two = f_temp[(f_temp['Datetime']>=two_start) & (f_temp['Datetime']<=two_end)][['Datetime']]
    times_two['Type'] = '2D'
    times_three = f_temp[(f_temp['Datetime']>=three_start) & (f_temp['Datetime']<=three_end)][['Datetime']]
    times_three['Type'] = '3D'
    times_df = pd.concat([times_two,times_three])
    times_df['Participant'] = participant

    type_times_df = pd.concat([type_times_df,times_df])
type_times_df = type_times_df.set_index(['Participant','Datetime'])  

In [59]:
type_times_df.head()

Type
Participant Datetime                
0001        2019-07-10 15:25:32   2D
            2019-07-10 15:25:34   2D
            2019-07-10 15:25:36   2D
            2019-07-10 15:25:38   2D
            2019-07-10 15:25:40   2D

In [60]:
feature_df.head()

ecg_heart_rate  ecg_hrv_std  ecg_hrv_rmssd  \
Participant Datetime                                                          
0001        2019-07-10 15:22:32       56.390977          NaN            NaN   
            2019-07-10 15:22:34       58.689424     1.625247    1043.373394   
            2019-07-10 15:22:36       58.843168     1.373544    1035.529128   
            2019-07-10 15:22:38       61.939107     2.277549    1019.231144   
            2019-07-10 15:22:40       61.907033     2.370811    1009.422396   

                                 ecg_hrv_lnrmssd  ppg_amplitude  \
Participant Datetime                                              
0001        2019-07-10 15:22:32              NaN            NaN   
            2019-07-10 15:22:34         6.950214            NaN   
            2019-07-10 15:22:36         6.942668       0.990295   
            2019-07-10 15:22:38         6.926804       0.976715   
            2019-07-10 15:22:40         6.917134       0.998230   

                                 skt_skin_temp   rsp_rate  eda_no_of_peaks  \
Participant Datetime                                                         
0001        2019-07-10 15:22:32      12.956869        NaN              2.0   
            2019-07-10 15:22:34      12.956752        NaN              2.0   
            2019-07-10 15:22:36      12.956626  22.865854              2.0   
            2019-07-10 15:22:38      12.956655  27.712759              2.0   
            2019-07-10 15:22:40      12.956718  21.755830              2.0   

                                 eda_low_power_spectrum  \
Participant Datetime                                      
0001        2019-07-10 15:22:32                0.008564   
            2019-07-10 15:22:34                0.009100   
            2019-07-10 15:22:36                0.009635   
            2019-07-10 15:22:38                0.010170   
            2019-07-10 15:22:40                0.010706   

                                 eda_high_power_spectrum  eda_high_low_ratio  \
Participant Datetime                                                           
0001        2019-07-10 15:22:32                 0.018872            0.453819   
            2019-07-10 15:22:34                 0.020051            0.453819   
            2019-07-10 15:22:36                 0.021231            0.453819   
            2019-07-10 15:22:38                 0.022410            0.453819   
            2019-07-10 15:22:40                 0.023590            0.453819   

                                 eda_mean  
Participant Datetime                       
0001        2019-07-10 15:22:32  0.001451  
            2019-07-10 15:22:34 -0.014861  
            2019-07-10 15:22:36 -0.008080  
            2019-07-10 15:22:38  0.002443  
            2019-07-10 15:22:40 -0.008870

In [61]:
features_complete = feature_df.merge(type_times_df, how = 'left', left_index = True, right_index = True)

In [62]:
features_complete.set_index('Type',append = True, inplace = True)

In [63]:
features_complete.head()

ecg_heart_rate  ecg_hrv_std  \
Participant Datetime            Type                                
0001        2019-07-10 15:22:32 NaN        56.390977          NaN   
            2019-07-10 15:22:34 NaN        58.689424     1.625247   
            2019-07-10 15:22:36 NaN        58.843168     1.373544   
            2019-07-10 15:22:38 NaN        61.939107     2.277549   
            2019-07-10 15:22:40 NaN        61.907033     2.370811   

                                      ecg_hrv_rmssd  ecg_hrv_lnrmssd  \
Participant Datetime            Type                                   
0001        2019-07-10 15:22:32 NaN             NaN              NaN   
            2019-07-10 15:22:34 NaN     1043.373394         6.950214   
            2019-07-10 15:22:36 NaN     1035.529128         6.942668   
            2019-07-10 15:22:38 NaN     1019.231144         6.926804   
            2019-07-10 15:22:40 NaN     1009.422396         6.917134   

                                      ppg_amplitude  skt_skin_temp   rsp_rate  \
Participant Datetime            Type                                            
0001        2019-07-10 15:22:32 NaN             NaN      12.956869        NaN   
            2019-07-10 15:22:34 NaN             NaN      12.956752        NaN   
            2019-07-10 15:22:36 NaN        0.990295      12.956626  22.865854   
            2019-07-10 15:22:38 NaN        0.976715      12.956655  27.712759   
            2019-07-10 15:22:40 NaN        0.998230      12.956718  21.755830   

                                      eda_no_of_peaks  eda_low_power_spectrum  \
Participant Datetime            Type                                            
0001        2019-07-10 15:22:32 NaN               2.0                0.008564   
            2019-07-10 15:22:34 NaN               2.0                0.009100   
            2019-07-10 15:22:36 NaN               2.0                0.009635   
            2019-07-10 15:22:38 NaN               2.0                0.010170   
            2019-07-10 15:22:40 NaN               2.0                0.010706   

                                      eda_high_power_spectrum  \
Participant Datetime            Type                            
0001        2019-07-10 15:22:32 NaN                  0.018872   
            2019-07-10 15:22:34 NaN                  0.020051   
            2019-07-10 15:22:36 NaN                  0.021231   
            2019-07-10 15:22:38 NaN                  0.022410   
            2019-07-10 15:22:40 NaN                  0.023590   

                                      eda_high_low_ratio  eda_mean  
Participant Datetime            Type                                
0001        2019-07-10 15:22:32 NaN             0.453819  0.001451  
            2019-07-10 15:22:34 NaN             0.453819 -0.014861  
            2019-07-10 15:22:36 NaN             0.453819 -0.008080  
            2019-07-10 15:22:38 NaN             0.453819  0.002443  
            2019-07-10 15:22:40 NaN             0.453819 -0.008870

In [64]:
features_complete.loc(axis = 0)['0002',:,'3D'].head(60)

ecg_heart_rate  ecg_hrv_std  \
Participant Datetime            Type                                
0002        2019-07-11 15:51:36 3D         78.879274     5.376291   
            2019-07-11 15:51:38 3D         72.747558     5.132690   
            2019-07-11 15:51:40 3D         70.585918     4.526805   
            2019-07-11 15:51:42 3D         74.406253     4.408948   
            2019-07-11 15:51:44 3D         74.477546     3.701160   
            2019-07-11 15:51:46 3D         73.298811     3.238235   
            2019-07-11 15:51:48 3D         78.210757     3.097391   
            2019-07-11 15:51:50 3D         74.829932     2.473164   
            2019-07-11 15:51:52 3D         71.770335     2.649906   
            2019-07-11 15:51:54 3D         71.656577     2.741515   
            2019-07-11 15:51:56 3D         73.072578     2.623842   
            2019-07-11 15:51:58 3D         73.692323     2.414450   
            2019-07-11 15:52:00 3D         74.184721     2.351044   
            2019-07-11 15:52:02 3D         72.047434     2.358602   
            2019-07-11 15:52:04 3D         76.403286     2.350859   
            2019-07-11 15:52:06 3D         78.370763     2.278289   
            2019-07-11 15:52:08 3D         76.142132     2.315033   
            2019-07-11 15:52:10 3D         72.356215     2.156801   
            2019-07-11 15:52:12 3D         73.664602     2.163369   
            2019-07-11 15:52:14 3D         76.102713     2.214147   
            2019-07-11 15:52:16 3D         78.363082     2.413515   
            2019-07-11 15:52:18 3D         78.451470     2.439017   
            2019-07-11 15:52:20 3D         74.838289     2.439040   
            2019-07-11 15:52:22 3D         76.612472     2.336027   
            2019-07-11 15:52:24 3D         79.512773     2.397410   
            2019-07-11 15:52:26 3D         77.844150     2.356061   
            2019-07-11 15:52:28 3D         75.376884     2.282004   
            2019-07-11 15:52:30 3D         76.297050     2.225257   
            2019-07-11 15:52:32 3D         77.922078     1.949887   
            2019-07-11 15:52:34 3D         79.854301     2.126836   
            2019-07-11 15:52:36 3D         75.516321     2.101949   
            2019-07-11 15:52:38 3D         72.909221     2.306936   
            2019-07-11 15:52:40 3D         76.294278     2.023631   
            2019-07-11 15:52:42 3D         80.473474     2.072258   
            2019-07-11 15:52:44 3D         79.961549     2.177619   
            2019-07-11 15:52:46 3D         80.317260     2.297793   
            2019-07-11 15:52:48 3D         78.542554     2.300666   
            2019-07-11 15:52:50 3D         77.461001     2.181753   
            2019-07-11 15:52:52 3D         77.895870     2.162834   
            2019-07-11 15:52:54 3D         79.491031     2.161572   
            2019-07-11 15:52:56 3D         81.398063     2.359102   
            2019-07-11 15:52:58 3D         81.081081     2.369117   
            2019-07-11 15:53:00 3D         78.080288     2.301939   
            2019-07-11 15:53:02 3D         77.264158     2.319528   
            2019-07-11 15:53:04 3D         75.956200     2.367156   
            2019-07-11 15:53:06 3D         80.477293     2.314978   
            2019-07-11 15:53:08 3D         85.296507     2.383921   
            2019-07-11 15:53:10 3D         77.331606     2.303161   
            2019-07-11 15:53:12 3D         74.740696     2.572532   
            2019-07-11 15:53:14 3D         80.512510     2.590809   
            2019-07-11 15:53:16 3D         86.462533     3.215091   
            2019-07-11 15:53:18 3D         85.132162     3.516004   
            2019-07-11 15:53:20 3D         74.641646     3.725147   
            2019-07-11 15:53:22 3D         73.840445     4.003557   
            2019-07-11 15:53:24 3D         77.771107     4.026771   
            2019-07-11 15:53:26 3D         78.776772     3.987410   
            2019-07-11 15:53:28 3D  

In [65]:
log_events.head()

context
Participant Datetime            Type                  
0001        2019-07-10 15:25:32 2D    StartFrontBuffer
            2019-07-10 15:27:32 2D     StopFrontBuffer
                                2D             StartLL
            2019-07-10 15:29:42 2D              StopLL
                                2D             StartHH

In [66]:
log_events.reset_index('Type', inplace = True)

In [67]:
temp_log = log_events.drop(["Type",'context'], axis = 1)
temp_log['temp'] = np.arange(0,len(log_events['context'].values))
temp_log.head()

temp
Participant Datetime                 
0001        2019-07-10 15:25:32     0
            2019-07-10 15:27:32     1
            2019-07-10 15:27:32     2
            2019-07-10 15:29:42     3
            2019-07-10 15:29:42     4

In [68]:
p_df = pd.DataFrame([])
for participant in participants:
    data = temp_log.loc(axis = 0)[participant]
    df = pd.DataFrame(data.resample('1s').sum())
    
    df['Participant'] = participant
    p_df = pd.concat([p_df,df])
p_df = p_df.reset_index().set_index(['Participant','Datetime'])

In [69]:
p_df.head()

temp
Participant Datetime                 
0001        2019-07-10 15:25:32     0
            2019-07-10 15:25:33     0
            2019-07-10 15:25:34     0
            2019-07-10 15:25:35     0
            2019-07-10 15:25:36     0

In [70]:
p_df = p_df.merge(log_events, how = 'left', right_index = True, left_index = True)

In [71]:
p_df['context'] = p_df['context'].fillna(method = 'ffill')
p_df['Type'] = p_df['Type'].fillna(method = 'ffill')
p_df.drop('temp', axis = 1, inplace = True)
# p_df.set_index("Type", append = True, inplace = True)

In [72]:
f = feature_df.merge(right = p_df, how = 'left', left_index= True, right_index = True)

In [73]:
f['context'] = f['context'].fillna(method = 'ffill')

In [74]:
f.head()

ecg_heart_rate  ecg_hrv_std  ecg_hrv_rmssd  \
Participant Datetime                                                          
0001        2019-07-10 15:22:32       56.390977          NaN            NaN   
            2019-07-10 15:22:34       58.689424     1.625247    1043.373394   
            2019-07-10 15:22:36       58.843168     1.373544    1035.529128   
            2019-07-10 15:22:38       61.939107     2.277549    1019.231144   
            2019-07-10 15:22:40       61.907033     2.370811    1009.422396   

                                 ecg_hrv_lnrmssd  ppg_amplitude  \
Participant Datetime                                              
0001        2019-07-10 15:22:32              NaN            NaN   
            2019-07-10 15:22:34         6.950214            NaN   
            2019-07-10 15:22:36         6.942668       0.990295   
            2019-07-10 15:22:38         6.926804       0.976715   
            2019-07-10 15:22:40         6.917134       0.998230   

                                 skt_skin_temp   rsp_rate  eda_no_of_peaks  \
Participant Datetime                                                         
0001        2019-07-10 15:22:32      12.956869        NaN              2.0   
            2019-07-10 15:22:34      12.956752        NaN              2.0   
            2019-07-10 15:22:36      12.956626  22.865854              2.0   
            2019-07-10 15:22:38      12.956655  27.712759              2.0   
            2019-07-10 15:22:40      12.956718  21.755830              2.0   

                                 eda_low_power_spectrum  \
Participant Datetime                                      
0001        2019-07-10 15:22:32                0.008564   
            2019-07-10 15:22:34                0.009100   
            2019-07-10 15:22:36                0.009635   
            2019-07-10 15:22:38                0.010170   
            2019-07-10 15:22:40                0.010706   

                                 eda_high_power_spectrum  eda_high_low_ratio  \
Participant Datetime                                                           
0001        2019-07-10 15:22:32                 0.018872            0.453819   
            2019-07-10 15:22:34                 0.020051            0.453819   
            2019-07-10 15:22:36                 0.021231            0.453819   
            2019-07-10 15:22:38                 0.022410            0.453819   
            2019-07-10 15:22:40                 0.023590            0.453819   

                                 eda_mean Type context  
Participant Datetime                                    
0001        2019-07-10 15:22:32  0.001451  NaN     NaN  
            2019-07-10 15:22:34 -0.014861  NaN     NaN  
            2019-07-10 15:22:36 -0.008080  NaN     NaN  
            2019-07-10 15:22:38  0.002443  NaN     NaN  
            2019-07-10 15:22:40 -0.008870  NaN     NaN

In [75]:
# f.reset_index('context', inplace = True)
s = [str(x).replace('Start','') for x in f['context'].values]
s = [str(x).replace('Stop','') for x in s]
s = [str(x).replace('nan','Garbage') for x in s]
f['context'] = s

In [76]:
f= f[f['context']!='Garbage']
f = f[f['context']!='FrontBuffer']
f = f[f['context']!='EndBuffer']


In [77]:
f.head()

ecg_heart_rate  ecg_hrv_std  ecg_hrv_rmssd  \
Participant Datetime                                                          
0001        2019-07-10 15:27:32       61.365192     4.851276     945.688088   
            2019-07-10 15:27:34       76.469010     5.652729     941.432840   
            2019-07-10 15:27:36       80.674995     7.013103     928.667268   
            2019-07-10 15:27:38       74.489475     7.375541     923.343304   
            2019-07-10 15:27:40       60.190684     6.974201     916.864589   

                                 ecg_hrv_lnrmssd  ppg_amplitude  \
Participant Datetime                                              
0001        2019-07-10 15:27:32         6.851913       0.998688   
            2019-07-10 15:27:34         6.847403       0.501251   
            2019-07-10 15:27:36         6.833751       0.998840   
            2019-07-10 15:27:38         6.828001       0.993958   
            2019-07-10 15:27:40         6.820960       0.998077   

                                 skt_skin_temp   rsp_rate  eda_no_of_peaks  \
Participant Datetime                                                         
0001        2019-07-10 15:27:32      12.957098  15.614582         9.133333   
            2019-07-10 15:27:34      12.957180  16.358931         9.266667   
            2019-07-10 15:27:36      12.957137  16.644474         9.400000   
            2019-07-10 15:27:38      12.957051  17.281312         9.533333   
            2019-07-10 15:27:40      12.961814  18.949808         9.666667   

                                 eda_low_power_spectrum  \
Participant Datetime                                      
0001        2019-07-10 15:27:32                0.073592   
            2019-07-10 15:27:34                0.072756   
            2019-07-10 15:27:36                0.071919   
            2019-07-10 15:27:38                0.071083   
            2019-07-10 15:27:40                0.070246   

                                 eda_high_power_spectrum  eda_high_low_ratio  \
Participant Datetime                                                           
0001        2019-07-10 15:27:32                 0.102356            0.718984   
            2019-07-10 15:27:34                 0.101977            0.713450   
            2019-07-10 15:27:36                 0.101598            0.707875   
            2019-07-10 15:27:38                 0.101220            0.702259   
            2019-07-10 15:27:40                 0.100841            0.696600   

                                 eda_mean Type context  
Participant Datetime                                    
0001        2019-07-10 15:27:32  4.610121   2D      LL  
            2019-07-10 15:27:34  4.538270   2D      LL  
            2019-07-10 15:27:36  4.479658   2D      LL  
            2019-07-10 15:27:38  4.433290   2D      LL  
            2019-07-10 15:27:40  4.371519   2D      LL

In [78]:
sensor = joblib.load('/mnt/shared_drive/data/brain_therapy/MVP_Focused_Attention/signal_with_events.pkl')

In [79]:
sensor.loc(axis = 0)['0002',:,:]

ecg_heart_rate  ecg_hrv_std  \
Participant Datetime            Type                                
0002        2019-07-11 15:53:35 3D         81.383972     4.047955   
            2019-07-11 15:53:36 3D         82.225693     4.078326   
            2019-07-11 15:53:38 3D         79.013030     3.858629   
            2019-07-11 15:53:40 3D         79.365079     3.793248   
            2019-07-11 15:53:42 3D         84.478486     3.633409   
            2019-07-11 15:53:44 3D         87.080537     3.977382   
            2019-07-11 15:53:46 3D         80.213904     3.707774   
            2019-07-11 15:53:48 3D         85.290964     3.721202   
            2019-07-11 15:53:50 3D         87.021382     3.624776   
            2019-07-11 15:53:52 3D         78.777654     3.042960   
            2019-07-11 15:53:54 3D         74.998125     3.381879   
            2019-07-11 15:53:56 3D         74.960047     3.748422   
            2019-07-11 15:53:58 3D         83.395453     3.773297   
            2019-07-11 15:54:00 3D         84.158104     3.788307   
            2019-07-11 15:54:02 3D         77.017787     3.967403   
            2019-07-11 15:54:04 3D         74.052941     4.384851   
            2019-07-11 15:54:06 3D         79.176749     4.384872   
            2019-07-11 15:54:08 3D         83.740405     4.432293   
            2019-07-11 15:54:10 3D         84.735183     4.514265   
            2019-07-11 15:54:12 3D         75.427799     4.657818   
            2019-07-11 15:54:14 3D         72.787267     4.714345   
            2019-07-11 15:54:16 3D         78.793774     4.717910   
            2019-07-11 15:54:18 3D         83.595380     4.590995   
            2019-07-11 15:54:20 3D         79.969854     4.101791   
            2019-07-11 15:54:22 3D        112.464855     9.551003   
            2019-07-11 15:54:24 3D         79.680709     9.406271   
            2019-07-11 15:54:26 3D         80.209389     9.237847   
            2019-07-11 15:54:28 3D         78.028139     9.281583   
            2019-07-11 15:54:30 3D         79.744592     9.264270   
            2019-07-11 15:54:32 3D         80.838850     9.190500   
...                                              ...          ...   
            2019-07-11 16:17:40 2D         70.681531     2.476701   
            2019-07-11 16:17:42 2D         67.636449     2.260691   
            2019-07-11 16:17:44 2D         68.657092     2.255325   
            2019-07-11 16:17:46 2D         58.877162     3.153948   
            2019-07-11 16:17:48 2D         67.796951     3.161347   
            2019-07-11 16:17:50 2D         71.508996     3.377859   
            2019-07-11 16:17:52 2D         69.911963     3.453255   
            2019-07-11 16:17:54 2D         70.142180     3.271966   
            2019-07-11 16:17:56 2D         71.196427     3.385394   
            2019-07-11 16:17:58 2D         69.185569     3.379143   
            2019-07-11 16:18:00 2D         72.102836     3.538113   
            2019-07-11 16:18:02 2D         66.142581     3.520746   
            2019-07-11 16:18:04 2D         69.133152     3.470132   
            2019-07-11 16:18:06 2D         74.131601     3.486939   
            2019-07-11 16:18:08 2D         74.600755     3.727583   
            2019-07-11 16:18:10 2D         76.809285     4.180772   
            2019-07-11 16:18:12 2D         83.248543     5.364587   
            2019-07-11 16:18:14 2D         75.704011     5.462156   
            2019-07-11 16:18:16 2D         66.140102     4.515942   
            2019-07-11 16:18:18 2D         67.443595     4.539457   
            2019-07-11 16:18:20 2D         67.126725     4.699564   
            2019-07-11 16:18:22 2D         69.296602     4.717392   
            2019-07-11 16:18:24 2D         68.032097     4.791450   
            2019-07-11 16:18:26 2D         70.485026     4.796627   
            2019-07-11 16:18:28 2D         61.978045     5.352909   
            2019-07-11 16:18:30 2D  

#### Replace Time with numbers

In [80]:
features = f.copy()

In [81]:
features = features.reset_index().set_index(["Participant","Type",'context',"Datetime"])

In [82]:
features.head(2)

ecg_heart_rate  ecg_hrv_std  \
Participant Type context Datetime                                           
0001        2D   LL      2019-07-10 15:27:32       61.365192     4.851276   
                         2019-07-10 15:27:34       76.469010     5.652729   

                                              ecg_hrv_rmssd  ecg_hrv_lnrmssd  \
Participant Type context Datetime                                              
0001        2D   LL      2019-07-10 15:27:32     945.688088         6.851913   
                         2019-07-10 15:27:34     941.432840         6.847403   

                                              ppg_amplitude  skt_skin_temp  \
Participant Type context Datetime                                            
0001        2D   LL      2019-07-10 15:27:32       0.998688      12.957098   
                         2019-07-10 15:27:34       0.501251      12.957180   

                                               rsp_rate  eda_no_of_peaks  \
Participant Type context Datetime                                          
0001        2D   LL      2019-07-10 15:27:32  15.614582         9.133333   
                         2019-07-10 15:27:34  16.358931         9.266667   

                                              eda_low_power_spectrum  \
Participant Type context Datetime                                      
0001        2D   LL      2019-07-10 15:27:32                0.073592   
                         2019-07-10 15:27:34                0.072756   

                                              eda_high_power_spectrum  \
Participant Type context Datetime                                       
0001        2D   LL      2019-07-10 15:27:32                 0.102356   
                         2019-07-10 15:27:34                 0.101977   

                                              eda_high_low_ratio  eda_mean  
Participant Type context Datetime                                           
0001        2D   LL      2019-07-10 15:27:32            0.718984  4.610121  
                         2019-07-10 15:27:34            0.713450  4.538270

In [83]:
sessions = ["2D","3D"]
events = features.index.get_level_values('context').unique()

In [84]:
features.head()

ecg_heart_rate  ecg_hrv_std  \
Participant Type context Datetime                                           
0001        2D   LL      2019-07-10 15:27:32       61.365192     4.851276   
                         2019-07-10 15:27:34       76.469010     5.652729   
                         2019-07-10 15:27:36       80.674995     7.013103   
                         2019-07-10 15:27:38       74.489475     7.375541   
                         2019-07-10 15:27:40       60.190684     6.974201   

                                              ecg_hrv_rmssd  ecg_hrv_lnrmssd  \
Participant Type context Datetime                                              
0001        2D   LL      2019-07-10 15:27:32     945.688088         6.851913   
                         2019-07-10 15:27:34     941.432840         6.847403   
                         2019-07-10 15:27:36     928.667268         6.833751   
                         2019-07-10 15:27:38     923.343304         6.828001   
                         2019-07-10 15:27:40     916.864589         6.820960   

                                              ppg_amplitude  skt_skin_temp  \
Participant Type context Datetime                                            
0001        2D   LL      2019-07-10 15:27:32       0.998688      12.957098   
                         2019-07-10 15:27:34       0.501251      12.957180   
                         2019-07-10 15:27:36       0.998840      12.957137   
                         2019-07-10 15:27:38       0.993958      12.957051   
                         2019-07-10 15:27:40       0.998077      12.961814   

                                               rsp_rate  eda_no_of_peaks  \
Participant Type context Datetime                                          
0001        2D   LL      2019-07-10 15:27:32  15.614582         9.133333   
                         2019-07-10 15:27:34  16.358931         9.266667   
                         2019-07-10 15:27:36  16.644474         9.400000   
                         2019-07-10 15:27:38  17.281312         9.533333   
                         2019-07-10 15:27:40  18.949808         9.666667   

                                              eda_low_power_spectrum  \
Participant Type context Datetime                                      
0001        2D   LL      2019-07-10 15:27:32                0.073592   
                         2019-07-10 15:27:34                0.072756   
                         2019-07-10 15:27:36                0.071919   
                         2019-07-10 15:27:38                0.071083   
                         2019-07-10 15:27:40                0.070246   

                                              eda_high_power_spectrum  \
Participant Type context Datetime                                       
0001        2D   LL      2019-07-10 15:27:32                 0.102356   
                         2019-07-10 15:27:34                 0.101977   
                         2019-07-10 15:27:36                 0.101598   
                         2019-07-10 15:27:38                 0.101220   
                         2019-07-10 15:27:40                 0.100841   

                                              eda_high_low_ratio  eda_mean  
Participant Type context Datetime                                           
0001        2D   LL      2019-07-10 15:27:32            0.718984  4.610121  
                         2019-07-10 15:27:34            0.713450  4.538270  
                         2019-07-10 15:27:36            0.707875  4.479658  
                         2019-07-10 15:27:38            0.702259  4.433290  
                         2019-07-10 15:27:40            0.696600  4.371519

In [85]:
t3 = pd.DataFrame([])
for participant in participants:
    t2 = pd.DataFrame([])
    counter = 1
    sessions = features.loc(axis = 0)[participant,:,:,:].index.get_level_values('Type').unique()
    for k, session in enumerate(sessions):
        t1 = pd.DataFrame([])
        events = features.loc(axis = 0)[participant,session,:,:].index.get_level_values('context').unique()
        for j,event in enumerate(events):
            d = features.loc(axis = 0)[participant,session,event,:]
            d['feature_time'] = np.arange(0,len(d['ecg_heart_rate'].values))
            d['event_order'] = j+1
            d['event_session_order'] = counter
            counter+=1
            t1 = pd.concat([t1,d])
        t1['session_order'] = k+1
#         print(k)
        t2 = pd.concat([t2,t1])
#         break
    t3 = pd.concat([t3,t2])
#     break

In [87]:
t3 = t3[t3['feature_time']<65]

In [88]:
t3.groupby(level = [0,1,2]).count()['ecg_heart_rate'].unique()

array([65])

In [89]:
joblib.dump(t3,'/mnt/shared_drive/data/brain_therapy/features_v2.pkl')

['/mnt/shared_drive/data/brain_therapy/features_v2.pkl']

In [ ]:
t33 = t3.reset_index()
t33.to_csv( "/mnt/bkt_prd_dsv_brain_therapy_raw/Features/Focused_Attention/features_v2.csv", index = False)

In [ ]:
features_complete = joblib.load('/mnt/shared_drive/data/brain_therapy/features.pkl')